# Filtragem Colaborativa Baseada em Item

In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

In [ ]:
# --- 1. Carregar os dados ---
path = "ds/"

customers_df = pd.read_csv(path+'olist_customers_dataset.csv')
geolocation_df = pd.read_csv(path+'olist_geolocation_dataset.csv')
orders_items_df = pd.read_csv(path+'olist_order_items_dataset.csv')
order_payments_df = pd.read_csv(path+'olist_order_payments_dataset.csv')
order_reviews_df = pd.read_csv(path+'olist_order_reviews_dataset.csv')
orders_df = pd.read_csv(path+'olist_orders_dataset.csv')
products_df = pd.read_csv(path+'olist_products_dataset.csv')
sellers_df = pd.read_csv(path+'olist_sellers_dataset.csv')
product_category_name_translation = pd.read_csv(path+'product_category_name_translation.csv')


In [4]:
print(
    f"orders_df: {orders_df.columns}\n"
    f"products_df: {products_df.columns}\n"
    f"customers_df: {customers_df.columns}\n"
    f"geolocation_df: {geolocation_df.columns}\n"
    f"orders_items_df: {orders_items_df.columns}\n"
    f"order_payments_df: {order_payments_df.columns}\n"
    f"order_reviews_df: {order_reviews_df.columns}\n"
    f"sellers_df: {sellers_df.columns}\n"
    f"product_category_name_translation: {product_category_name_translation.columns}\n"
)

orders_df: Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')
products_df: Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')
customers_df: Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')
geolocation_df: Index(['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng',
       'geolocation_city', 'geolocation_state'],
      dtype='object')
orders_items_df: Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')
order_payments_df: Index(['or

In [ ]:
# --- 2. Preparar os dados para o modelo  ---
orders_customers_df = orders_df.merge(customers_df, on='customer_id')

# Agora, vamos unir o resultado com products_df e orders_items_df to get product_id and other relevant columns
data_merged = orders_customers_df.merge(orders_items_df, on='order_id').merge(products_df, on='product_id')

# Passo 2.2: Contar a frequência de compra de cada produto por cada cliente
product_counts = data_merged.groupby(['customer_id', 'product_id']).size().reset_index(name='purchase_count')

# Passo 2.3: Criar a matriz de interações (usuário x item)
user_item_matrix = product_counts.pivot_table(
    index='customer_id',
    columns='product_id',
    values='purchase_count'
).fillna(0)

# Passo 2.4: Unir as informações de produto com a tabela de tradução
products_with_names = products_df.merge(product_category_name_translation, on='product_category_name', how='left')

os.system('say "bip"')

/var/folders/96/wgrm9ygd6qb8n9h5g15qynww0000gq/T/ipykernel_19719/2751491204.py:13: PerformanceWarning: The following operation may generate 3251143366 cells in the resulting pandas object.
  user_item_matrix = product_counts.pivot_table(


0

In [ ]:
# --- 3. Treinar o modelo de recomendação (Filtragem Colaborativa) ---
# Calcular a similaridade do cosseno entre os produtos
item_similarity = cosine_similarity(user_item_matrix.T)

# Converter a matriz de similaridade para um DataFrame
item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)

0

In [13]:
# --- 4. Função de Recomendação ---
def get_recommendations(product_id, n=5):
    """
    Retorna os N produtos mais similares a um produto dado.
    """
    if product_id not in item_similarity_df.columns:
        return "Produto não encontrado no conjunto de dados de treino."

    # Obter a série de similaridade para o produto
    similar_scores = item_similarity_df[product_id]

    # Ordenar os produtos pela similaridade e pegar os N mais próximos
    similar_products_ids = similar_scores.sort_values(ascending=False).index[1:n+1].tolist()
    
    # Adicionar o nome da categoria para uma melhor visualização
    recommended_info = products_with_names[products_with_names['product_id'].isin(similar_products_ids)]
    
    return recommended_info[['product_id', 'product_category_name']]

In [ ]:
# --- 5. Exemplo de uso ---
example_product_id = '37dffde0476450e75dae423136954b0f'

# Buscar o nome da categoria do produto
product_row = products_with_names[products_with_names['product_id'] == example_product_id]
if not product_row.empty:
    product_name = product_row.iloc[0]['product_category_name']
else:
    product_name = 'Categoria desconhecida'

print(f"Recomendações para o produto: {example_product_id} ({product_name})")

recommendations = get_recommendations(example_product_id, n=5)

if isinstance(recommendations, str):
    print(recommendations)
else:
    print(recommendations)

Recomendações para o produto: 37dffde0476450e75dae423136954b0f (automotivo)
                             product_id product_category_name
318    8a2d3285de64b2117fbf83784acd09fc            automotivo
3508   ab9730d0380f2a4bb056dbed9417478a            brinquedos
18874  ab959a147167f22f3e0c05eb06845771           eletronicos
23971  aba0c26e6fcc5a413da7fcfad4b04ac1            perfumaria
31688  00066f42aeeb9f3007548bb9d3f33c38            perfumaria


# Filtragem Colaborativa Baseada em Usuário

In [8]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Carregar dados
orders = pd.read_csv('ds/olist_orders_dataset.csv')
order_items = pd.read_csv('ds/olist_order_items_dataset.csv')

# Matriz usuário x item: linhas = clientes, colunas = produtos, valores = quantidade de compras
user_item = order_items.merge(orders, on='order_id')
user_item_matrix = user_item.pivot_table(
    index='customer_id',
    columns='product_id',
    values='order_item_id',
    aggfunc='count'
).fillna(0)

# Similaridade entre usuários
user_similarity = cosine_similarity(user_item_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

# Função de recomendação baseada em usuário
def recommend_by_user(customer_id, n=5):
    if customer_id not in user_similarity_df.index:
        return "Usuário não encontrado."
    similar_users = user_similarity_df[customer_id].sort_values(ascending=False).index[1:n+1]
    products = user_item[user_item['customer_id'].isin(similar_users)]['product_id'].value_counts().index.tolist()
    return products[:n]

# Exemplo de uso: recomendações para o primeiro cliente da matriz
print(recommend_by_user(user_item_matrix.index[0]))

/var/folders/96/wgrm9ygd6qb8n9h5g15qynww0000gq/T/ipykernel_22730/2696362612.py:10: PerformanceWarning: The following operation may generate 3251143366 cells in the resulting pandas object.
  user_item_matrix = user_item.pivot_table(


: 

# Recomendação Baseada em Conteúdo

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

products = pd.read_csv('ds/olist_products_dataset.csv')

# Usando apenas a categoria como conteúdo
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(products['product_category_name'].fillna(''))

cos_sim = cosine_similarity(tfidf_matrix)
cos_sim_df = pd.DataFrame(cos_sim, index=products['product_id'], columns=products['product_id'])

def recommend_by_content(product_id, n=5):
    if product_id not in cos_sim_df.columns:
        return "Produto não encontrado."
    similar_products = cos_sim_df[product_id].sort_values(ascending=False).index[1:n+1]
    return similar_products.tolist()

print(recommend_by_content(products['product_id'].iloc[0]))

['69918318f27e9853c6d2a68906018a14', '0de1e69dff6f40f9f71be7034a8bb8d2', 'ca3d8213acc967dd4e7c85c05406f71f', '9fa1e97015f5c5e76add50079bb0ab44', '2ada60128014f639dffa4a481cd1255a']


# Recomendação por Popularidade

In [3]:
# Produtos mais vendidos
popular_products = order_items['product_id'].value_counts().head(5)
print("Mais vendidos:", popular_products.index.tolist())

Mais vendidos: ['aca2eb7d00ea1a7b8ebd4e68314663af', '99a4788cb24856965c36a24e339b6058', '422879e10f46682990de24d770e7f83d', '389d119b48cf3043d311335e499d9c6b', '368c6c730842d78016ad823897a372db']


# Recomendação Personalizada por Perfil

In [4]:
# Exemplo: produtos mais comprados por clientes de SP
customers = pd.read_csv('ds/olist_customers_dataset.csv')
orders_customers = orders.merge(customers, on='customer_id')
sp_orders = orders_customers[orders_customers['customer_state'] == 'SP']
sp_items = order_items[order_items['order_id'].isin(sp_orders['order_id'])]
sp_popular = sp_items['product_id'].value_counts().head(5)
print("Mais populares em SP:", sp_popular.index.tolist())

Mais populares em SP: ['aca2eb7d00ea1a7b8ebd4e68314663af', '99a4788cb24856965c36a24e339b6058', '422879e10f46682990de24d770e7f83d', '368c6c730842d78016ad823897a372db', '389d119b48cf3043d311335e499d9c6b']


# Recomendação por Sazonalidade/Tempo

In [5]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['month'] = orders['order_purchase_timestamp'].dt.month
# Produtos mais vendidos em dezembro
dez_orders = orders[orders['month'] == 12]
dez_items = order_items[order_items['order_id'].isin(dez_orders['order_id'])]
dez_popular = dez_items['product_id'].value_counts().head(5)
print("Mais vendidos em dezembro:", dez_popular.index.tolist())

Mais vendidos em dezembro: ['422879e10f46682990de24d770e7f83d', '4c2394abfbac7ff59ec7a420918562fa', 'aca2eb7d00ea1a7b8ebd4e68314663af', '368c6c730842d78016ad823897a372db', '53759a2ecddad2bb87a079a1f1519f73']


# Recomendação Baseada em Avaliações

In [6]:
reviews = pd.read_csv('ds/olist_order_reviews_dataset.csv')
order_items_reviews = order_items.merge(reviews, on='order_id')
best_rated = order_items_reviews.groupby('product_id')['review_score'].mean().sort_values(ascending=False).head(5)
print("Produtos melhor avaliados:", best_rated.index.tolist())

Produtos melhor avaliados: ['00066f42aeeb9f3007548bb9d3f33c38', '60314cbf527af04ebc935be7c02dc1cd', '5fe23e6307c231e287f13c0a4be06c88', '5fe490e61e7a37f2f0d1ad1771ac027d', 'c696cd2db7870be08c37822f7c0ef831']


# Recomendação Cross-sell (produtos comprados juntos)

In [7]:
# Produtos frequentemente comprados juntos
from itertools import combinations
from collections import Counter

order_products = order_items.groupby('order_id')['product_id'].apply(list)
pairs = Counter()
for products in order_products:
    for combo in combinations(sorted(products), 2):
        pairs[combo] += 1
print("Pares de produtos mais comprados juntos:", pairs.most_common(5))

Pares de produtos mais comprados juntos: [(('422879e10f46682990de24d770e7f83d', '422879e10f46682990de24d770e7f83d'), 279), (('ee3d532c8a438679776d222e997606b3', 'ee3d532c8a438679776d222e997606b3'), 195), (('9571759451b1d780ee7c15012ea109d4', '9571759451b1d780ee7c15012ea109d4'), 190), (('a62e25e09e05e6faf31d90c6ec1aa3d1', 'a62e25e09e05e6faf31d90c6ec1aa3d1'), 141), (('368c6c730842d78016ad823897a372db', '368c6c730842d78016ad823897a372db'), 139)]
